In [ ]:
import optuna
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

def objective_catboost(trial, X, y, task_type='classification'):
    """
    Функция для оптимизации гиперпараметров CatBoost
    """
    # Параметры для оптимизации
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000), # МЕНЯЙ НА
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0.1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 10),
        'verbose': False,
        'random_seed': 42,
        'task_type': 'GPU'
    }
    
    # Разделение данных
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Выбор модели в зависимости от задачи
    if task_type == 'classification':
        model = CatBoostClassifier(**params)
        metric = accuracy_score
    else:  # regression
        model = CatBoostRegressor(**params)
        metric = mean_squared_error
    
    # Обучение модели
    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Предсказание и оценка
    y_pred = model.predict(X_val)
    score = metric(y_val, y_pred)
    
    # Для регрессии используем отрицательный MSE (Optuna максимизирует)
    if task_type == 'regression':
        score = -score
    
    return score

# Пример использования
def optimize_catboost(X, y, task_type='classification', n_trials=100):
    study = optuna.create_study(direction='maximize')
    study.optimize(
        lambda trial: objective_catboost(trial, X, y, task_type), 
        n_trials=n_trials
    )
    
    print("Лучшие параметры CatBoost:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    print(f"Лучшее значение метрики: {study.best_value}")
    
    return study.best_params

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

def objective_lightgbm(trial, X, y, task_type='classification'):
    """
    Функция для оптимизации гиперпараметров LightGBM
    """
    # Параметры для оптимизации
    params = {
        'objective': 'binary' if task_type == 'classification' else 'regression',
        'metric': 'binary_logloss' if task_type == 'classification' else 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'random_state': 42
    }
    
    # Разделение данных
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Создание датасетов LightGBM
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
    # Обучение модели
    model = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)]
    )
    
    # Предсказание и оценка
    y_pred = model.predict(X_val)
    
    if task_type == 'classification':
        y_pred_binary = (y_pred > 0.5).astype(int)
        score = accuracy_score(y_val, y_pred_binary)
    else:  # regression
        score = -mean_squared_error(y_val, y_pred)  # отрицательный MSE
    
    return score

def optimize_lightgbm(X, y, task_type='classification', n_trials=100):
    study = optuna.create_study(direction='maximize')
    study.optimize(
        lambda trial: objective_lightgbm(trial, X, y, task_type), 
        n_trials=n_trials
    )
    
    print("Лучшие параметры LightGBM:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    print(f"Лучшее значение метрики: {study.best_value}")
    
    return study.best_params

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

def objective_xgboost(trial, X, y, task_type='classification'):
    """
    Функция для оптимизации гиперпараметров XGBoost
    """
    # Параметры для оптимизации
    params = {
        'verbosity': 0,
        'objective': 'binary:logistic' if task_type == 'classification' else 'reg:squarederror',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': 42
    }
    
    if params['booster'] == 'gbtree' or params['booster'] == 'dart':
        params['max_depth'] = trial.suggest_int('max_depth', 3, 10)
        params['min_child_weight'] = trial.suggest_int('min_child_weight', 1, 10)
    
    # Разделение данных
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Обучение модели
    model = xgb.XGBClassifier(**params) if task_type == 'classification' else xgb.XGBRegressor(**params)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Предсказание и оценка
    y_pred = model.predict(X_val)
    
    if task_type == 'classification':
        score = accuracy_score(y_val, y_pred)
    else:  # regression
        score = -mean_squared_error(y_val, y_pred)  # отрицательный MSE
    
    return score

def optimize_xgboost(X, y, task_type='classification', n_trials=100):
    study = optuna.create_study(direction='maximize')
    study.optimize(
        lambda trial: objective_xgboost(trial, X, y, task_type), 
        n_trials=n_trials
    )
    
    print("Лучшие параметры XGBoost:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    print(f"Лучшее значение метрики: {study.best_value}")
    
    return study.best_params